In [2]:
import pandas as pd

数据预处理：
1. 上午时间不变，下午的时间+12
2. 时间里的毫秒去掉
3. 时间读取到pandas内 dates = pd.to_datetime(date_strings, format="%d-%m月 -%y %H.%M.%S.%f")
4. 因为数据只给了同一个月份的，暂且先不考虑跨月和跨年，先考虑跨日
5. 操作内容前缀去掉

In [6]:
class DataProcess:
    def __init__(self, path: str) -> None:
        self.data = pd.read_csv(path)
    
    def process(self):
        self.data = self._data_process(self.data)
        return self.data
    
    @staticmethod
    def _data_process(df: pd.DataFrame):
        df_new = pd.DataFrame()
        data_time_format = "%d-%m月 -%y %H.%M.%S.%f"
        # 拆分出时间和上下午
        df_new['日期时间'] = pd.to_datetime(df['操作时间'].str.rsplit(' ', n=1, expand=True)[0], format=data_time_format)
        # 添加12小时给下午的日期时间
        pm_rows = df['操作时间'].str.endswith("下午")
        time_condition = (df_new['日期时间'].dt.hour != 12)
        df_new.loc[pm_rows & time_condition, '日期时间'] += pd.to_timedelta(12, unit='h')

        df_new['账号'] = df['从账号'].copy()
        df_new['机构'] = df['机构'].copy()
        df_new['操作内容'] = df['操作内容'].str.slice(36, None).str.strip()
        df_new['源IP'] = df['源IP'].copy()
        df_new = df_new.sort_values(by='日期时间', ascending=True)

        df_new.to_csv('data_process.csv')

        return df_new

df = DataProcess('data.csv').process()
df

,日期时间,账号,机构,操作内容,源IP
109868,2023-05-01 08:21:01,29527d9c-1b55-4121-9450-1acc95dbaa62,榆林全市,用户登陆,10.136.71.74
109867,2023-05-01 08:21:02,29527d9c-1b55-4121-9450-1acc95dbaa62,榆林全市,访问首页模块,10.136.71.74
109866,2023-05-01 08:21:08,29527d9c-1b55-4121-9450-1acc95dbaa62,榆林全市,访问5GSA驻留比模块,10.136.71.74
109864,2023-05-01 08:22:54,1c7cc983-5870-4892-af7b-c06a86ce59d7,延安全市,用户登陆,10.136.50.122
109865,2023-05-01 08:22:55,1c7cc983-5870-4892-af7b-c06a86ce59d7,延安全市,访问首页模块,10.136.50.122
...,...,...,...,...,...
5,2023-05-31 23:00:11,dc57ae05-2e96-4866-bffe-501deb9ffb01,产品运营中心,访问首页模块,10.136.14.86
3,2023-05-31 23:00:51,dc57ae05-2e96-4866-bffe-501deb9ffb01,产品运营中心,用户登陆,10.136.14.86
2,2023-05-31 23:00:52,dc57ae05-2e96-4866-bffe-501deb9ffb01,产品运营中心,访问首页模块,10.136.14.86
1,2023-05-31 23:01:00,dc57ae05-2e96-4866-bffe-501deb9ffb01,产品运营中心,访问智能家居质量日报模块,10.136.14.86


分别定义五个类：

业务高频访问：为每一个业务维护一个字典，{账户:访问次数, 账户:访问次数……}，存储不同账号访问的次数，每隔1分钟清零，字典值超过5打上业务高频访问标签。

账号复用：为每一个账号维护一个集合，保存此账号每次登录时的ip和时间，两分钟内如果集合len>=3，即账号复用，每隔2分钟清零。

非常用IP访问：TODO:这个是否要一次读完全部日志？一条一条读无法判断之后还会不会来。

非工作时间访问：不在8:00~19:00之间的日志都是。

登录异常： 为每个账号维护一个字典，记录访问次数，字典值>=8打上登录异常，一分钟清零

         为每个账号维护一个集合，保存此账号登录的ip，一分钟内若集合len>=2视为登录异常

---


清零操作：每次读到一行数据，如果是一分钟清零，如果新数据的分钟数 = 原来数据分钟数+1，将原数据结构清零。

         两分钟清零就是如果读到了新数据分钟数 = min(原始数据分钟数)+2



优化空间：
1. 五个操作共同维护数据结构，{账号1:{访问次数:{}}, 账号2：{}} 
2. 登录异常1分钟2个ip和2分钟三个ip的逻辑能不能合在一起，只用一个字典来处理。
3. 

为了减少损耗，一次读取的时候同时进行五个操作，因此类里的每个detection方法读取的参数是一行Series

业务高频访问

In [ ]:
from datetime import timedelta


class User:
    __slots__ = ('time', 'business')
    def __init__(self,timestamp):
        self.time=timestamp
        self.business={}


class HighFrequencyAccess:
    def __init__(self) -> None:
        self.user_dict = {}

    def detection(self,row) -> bool:
        user_id= row['账号']
        timestamp = row['日期时间']
        transaction  = row['操作内容']
        if user_id in self.user_dict:
            user = self.user_dict[user_id]
            if timestamp-user.time >= timedelta(minutes=1) or timestamp.minute!=user.time.minute:
                del self.user_dict[user_id]
                # 删除之前的数据
                self.user_dict[user_id] = User(timestamp)

            user = self.user_dict[user_id]
        else:
            user = User(timestamp)
        user.business[transaction] = user.business.get(transaction, 0) + 1
        self.user_dict[user_id] = user
        
        if user.business[transaction]>5:
            return True
        else :
            return False

账号复用

In [7]:
from datetime import timedelta

class AccountReuse:
    def __init__(self) -> None:
        self.account_data = {}

    def detection(self, row):
        account = row['账号']
        ip = row['源IP']
        timestamp = row['日期时间']

        # 处理新账户
        if account not in self.account_data:
            self.account_data[account] = {'ips': set(), 'last_timestamp': timestamp}
        
        # 每2分钟清空一个ips集合
        if (timestamp - self.account_data[account]['last_timestamp']) > timedelta(minutes=2):
            self.account_data[account]['ips'] = set()

        self.account_data[account]['ips'].add(ip)
        self.account_data[account]['last_timestamp'] = timestamp

        if len(self.account_data[account]['ips']) >= 3:
            print(f"账号 {account} 在时间 {timestamp} 存在账号复用行为")

In [12]:
account_reuse = AccountReuse()
account_reuse.detection(df)

TypeError: unhashable type: 'Series'

非常用IP访问

In [5]:
class UserData:
    pass


class UncommonIp:
    def __init__(self):
        self.account_data = {}

    def detection(self, df):
        for index, row in df.iterrows():
            account = row['账号']
            ip = row['源IP']
            timestamp = row['日期时间']

            if account not in self.account_data:
                self.account_data[account] = {'ips': set(), 'ip_counts': {}, 'total_ops': 0}
            
            self.account_data[account]['ips'].add(ip)
            self.account_data[account]['ip_counts'][ip] = self.account_data[account]['ip_counts'].get(ip, 0) + 1
            self.account_data[account]['total_ops'] += 1

            if self.account_data[account]['total_ops'] > 100:
                uncommon_ips = [ip for ip, count in self.account_data[account].items() if count < 5]
                if len(uncommon_ips) > 0:
                    print(f"账号 {account} 在时间 {timestamp} 存在异常IP {uncommon_ips}")

In [6]:
df = DataProcess(df_new).process()
account_reuse = UncommonIp()
account_reuse.detection(df)

NameError: name 'DataProcess' is not defined

✅ 非工作时间访问

In [51]:
class OutOfHoursAccess:
    def __init__(self, start, end) -> None:
        self.start = start
        self.end = end

    def detection(self, row: pd.core.series.Series) -> bool:
        datetime_value = row['日期时间'].time()
        # TODO: 19:00:00算不算？
        if not (self.start <= datetime_value <= self.end):
            # print(f"Row {row}: {datetime_value} is not in the range.")
            return True
        return False


登录异常

In [ ]:
class LoginException:
    def __init__(self) -> None:
        pass

    def detection(self) -> int:
        pass

main函数

In [ ]:
# 实例化非工作时间访问检查器
start = pd.to_datetime('08:00:00').time()
end = pd.to_datetime('19:00:00').time()

highfrequency = HighFrequencyAccess()
accountreuse = AccountReuse()
uncommonip = UncommonIp()
outHour = OutOfHoursAccess(start, end)
login = LoginException()

for i, row in df.iterrows():
    outHour.detection(row)